In [ ]:
import os

import pandas as pd

# Parameters

In [ ]:
database_files = snakemake.input.database_files
enrichment_files = snakemake.input.enrichment_files

fname_out = snakemake.output.fname

# Load data

In [ ]:
df_list = []
for fname in database_files:
    tmp = pd.read_csv(fname)
    
    data_source = os.path.basename(fname).split('.')[1]
    tmp['tad_source'] = data_source
    
    df_list.append(tmp)

df_data = pd.DataFrame(tmp)
df_data.sort_values(['diseaseId', 'snpId']).head()

In [ ]:
df_list = []
for fname in enrichment_files:
    tmp = pd.read_csv(fname)
    
    data_source = os.path.basename(fname).split('.')[1]
    tmp['tad_source'] = data_source
    
    df_list.append(tmp)

df_enr = pd.DataFrame(tmp)
df_enr.head()

# Merge data

In [ ]:
df_data_trans = pd.melt(
    df_data,
    id_vars=[x for x in df_data.columns if not x.startswith('TAD_')],
    var_name='TAD_type', value_name='TAD_relation')

df_data_trans['TAD_type'] = df_data_trans['TAD_type'].str[len('TAD_'):]

df_data_trans.head()

In [ ]:
df_merged = df_data_trans.merge(
    df_enr, how='outer')

In [ ]:
df_merged.sort_values(['TAD_type', 'diseaseId', 'snpId'], inplace=True)

In [ ]:
df_merged

# Save results

In [ ]:
df_merged.to_csv(fname_out, index=False)